In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
!pip install cleverhans==3.0.1 tensorflow==1.8.0

     |████████████████████████████████| 204kB 3.5MB/s 
     |████████████████████████████████| 49.1MB 1.7MB/s 
     |████████████████████████████████| 51kB 21.9MB/s 
     |████████████████████████████████| 163kB 49.7MB/s 
     |████████████████████████████████| 3.1MB 32.4MB/s 
     |████████████████████████████████| 890kB 37.5MB/s 
  Created wheel for html5lib: filename=html5lib-0.9999999-cp36-none-any.whl size=107221 sha256=36abda1bb36682ddf7edad93c1098ad55dd8b7411455b88c26c9b7c2c63690c0
  Stored in directory: /root/.cache/pip/wheels/50/ae/f9/d2b189788efcf61d1ee0e36045476735c838898eef1cad6e29
Successfully built html5lib
ERROR: magenta 0.3.19 has requirement tensorflow>=1.12.0, but you'll have tensorflow 1.8.0 which is incompatible.
  Found existing installation: html5lib 1.0.1
    Uninstalling html5lib-1.0.1:
      Successfully uninstalled html5lib-1.0.1
  Found existing installation: bleach 3.1.0
    Uninstalling bleach-3.1.0:
      Successfully uninstalled bleach-3.1.0
  Found exist

In [0]:
home_dir = '/content/gdrive/My Drive/Colab Notebooks/newlogic'
import sys
sys.path.append(home_dir)

In [0]:
# model_path = home_dir + '/output/cnn-mnist-relu.pkl'
# model_path = home_dir + '/output/cnn-mnist-relu-kernel.pkl'
# model_path = home_dir + '/output/cnn-mnist-relog.pkl'
# model_path = home_dir + '/output/cnn-mnist-relu-kernel.pkl'
# model_path = home_dir + '/output/cnn-mnist-relu-kernel-sigmoid_out-strictening_1.pkl'
# model_path = home_dir + '/output/cnn-mnist-relog-kernel-minmaxout_4_2-sigmoid-out-strictening_1.pkl'
# model_path = home_dir + '/output/cnn-mnist-relog-kernel-minmaxout_4_2-sigmoid-out.pkl'

In [0]:
from cleverhans.attacks import SaliencyMapMethod
from cleverhans.dataset import MNIST
from cleverhans.loss import CrossEntropy
from cleverhans.utils import other_classes, set_log_level
from cleverhans.utils import pair_visual, grid_visual, AccuracyReport
from cleverhans.utils_tf import model_eval, model_argmax
from cleverhans.train import train
from cleverhans.utils_pytorch import convert_pytorch_model_to_tf
from cleverhans.model import CallableModelWrapper
from cleverhans.attacks import CarliniWagnerL2

In [0]:
import numpy as np
from six.moves import xrange
import tensorflow as tf
import torch
from torchvision import datasets, transforms

In [0]:
import pandas as pd

In [8]:
# Create TF session and set as Keras backend session
sess = tf.Session()
print("Created TensorFlow session.")

Created TensorFlow session.


In [0]:
nb_classes = 10
source_samples = 100
attack_iterations = 1000

VIZ_ENABLED = viz_enabled = True
BATCH_SIZE = 128
NB_EPOCHS = 6
SOURCE_SAMPLES = 10
LEARNING_RATE = .001
CW_LEARNING_RATE = .2
ATTACK_ITERATIONS = 100
TARGETED = targeted = False

In [0]:
def evaluate_model(model_path):
  print('Evaluating model: ' + model_path)

  torch_model_orig = torch.load(model_path)
  torch_model = lambda x: torch_model_orig(x)[0] # to standard format
  tf_model_fn = convert_pytorch_model_to_tf(torch_model, out_dims=10)    
  cleverhans_model = CallableModelWrapper(tf_model_fn, output_layer='logits')
  cw = CarliniWagnerL2(cleverhans_model, sess=sess)
  test_data = datasets.MNIST(home_dir + '/mnist', train=False, transform=transforms.ToTensor())
  test_loader = torch.utils.data.DataLoader(test_data, batch_size=source_samples)
  x_test, y_test = next(iter(test_loader))
  y_test_one_hot = torch.nn.functional.one_hot(y_test, 10).cpu().detach().numpy()  
  nchannels, img_rows, img_cols = x_test.shape[1:]
  x = tf.placeholder(tf.float32, shape=(None, nchannels, img_rows, img_cols))
  y = tf.placeholder(tf.float32, shape=(None, nb_classes))
  preds = cleverhans_model.get_logits(x)

  # Keep track of success (adversarial example classified in target)
  results = np.zeros((nb_classes, source_samples), dtype='i')

  # Rate of perturbed features for each test set example and target class
  perturbations = np.zeros((nb_classes, source_samples), dtype='f')

  cw_params_batch_size = source_samples
  adv_ys = None
  cw_params = {'binary_search_steps': 1,
              'y': adv_ys,
              'max_iterations': attack_iterations,
              'learning_rate': CW_LEARNING_RATE,
              'batch_size': cw_params_batch_size,
              'initial_const': 10}

  adv = cw.generate_np(x_test, **cw_params)

  eval_params = {'batch_size': np.minimum(nb_classes, source_samples)}
  if targeted:
    acc = model_eval(sess, x, y, preds, adv, adv_ys, args=eval_params)
  else:
    acc = model_eval(sess, x, y, preds, adv, y_test_one_hot, args=eval_params)
  print('Accuracy: %.3f' % acc)
  return acc

In [0]:
models = pd.read_csv(home_dir + '/mnist-models.csv')

In [12]:
models.tail()

,bias_l1,bias_l2,conv1_out_channels,conv2_out_channels,l1,l2,max_folding_factor,min_folding_factor,name,regularization,use_maxout,use_quadratic_kernel,use_relog,use_sigmoid_out
8,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,cnn-mnist-relog-maxout_2,NaN,max,NaN,True,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,2.0,4.0,cnn-mnist-relog-minmaxout_4_2,NaN,minmax,NaN,True,NaN
10,NaN,NaN,NaN,NaN,NaN,NaN,2.0,4.0,cnn-mnist-relog-minmaxout_4_2-sigmoid_out,NaN,minmax,NaN,True,True
11,NaN,NaN,NaN,NaN,NaN,0.1,2.0,4.0,cnn-mnist-relog-minmaxout_4_2-sigmoid_out-max_...,max-margin,minmax,NaN,True,True
12,NaN,NaN,NaN,NaN,NaN,0.1,2.0,4.0,cnn-mnist-relog-kernel-minmaxout_4_2-sigmoid_o...,max-margin,minmax,True,True,True


In [30]:
results = []
for model_name in models.name:
  model_path = home_dir + '/output/' + model_name + '.pkl'
  accuracy = evaluate_model(model_path)
  results.append({
      'model_name': model_name,
      'accuracy': accuracy,
      'attack_type': 'C&W',
  })

[INFO 2019-10-19 01:15:12,116 cleverhans] Constructing new graph for attack CarliniWagnerL2


Evaluating model: /content/gdrive/My Drive/Colab Notebooks/newlogic/output/cnn-mnist-relu.pkl


[INFO 2019-10-19 01:15:19,880 cleverhans] Constructing new graph for attack CarliniWagnerL2


Accuracy: 0.000
Evaluating model: /content/gdrive/My Drive/Colab Notebooks/newlogic/output/cnn-mnist-relu-kernel.pkl


[INFO 2019-10-19 01:15:28,568 cleverhans] Constructing new graph for attack CarliniWagnerL2


...

Accuracy: 0.010
Evaluating model: /content/gdrive/My Drive/Colab Notebooks/newlogic/output/cnn-mnist-relog-minmaxout_4_2.pkl


[INFO 2019-10-19 01:16:57,558 cleverhans] Constructing new graph for attack CarliniWagnerL2


Accuracy: 0.020
Evaluating model: /content/gdrive/My Drive/Colab Notebooks/newlogic/output/cnn-mnist-relog-minmaxout_4_2-sigmoid_out.pkl
Accuracy: 0.190
Evaluating model: /content/gdrive/My Drive/Colab Notebooks/newlogic/output/cnn-mnist-relog-minmaxout_4_2-sigmoid_out-max_margin.pkl


[INFO 2019-10-19 01:17:25,732 cleverhans] Constructing new graph for attack CarliniWagnerL2


Accuracy: 0.310


In [0]:
pd.DataFrame(results).to_csv(home_dir + '/output/mnist-cnw-results.csv', index=False)

In [14]:
results = []
for model_name in models.name.iloc[-1:]:
    evaluate_model(home_dir + '/output/' + model_name + '.pkl')

Evaluating model: /content/gdrive/My Drive/Colab Notebooks/newlogic/output/cnn-mnist-relog-kernel-minmaxout_4_2-sigmoid_out-max_margin.pkl


[INFO 2019-10-19 13:59:40,200 cleverhans] Constructing new graph for attack CarliniWagnerL2


Accuracy: 0.560
